In [2]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained("models/gemma-2b.gguf", device_map="tpu:0",
                                         from_type="gemma",
                                         load_eager=True
                                         )

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("alpindale/gemma-2b")
tokenizer.padding_side = "right"

In [5]:
# import jax.numpy as jnp
# import jax
# from micrlhf.llama import LlamaBlock

# tokens = tokenizer.encode("The quick brown fox jumps over the lazy dog")
# get_resids = llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda i, x:
#     pz.nn.Sequential([
#         pz.de.TellIntermediate.from_config(tag=f"resid_pre_{i}"),
#         x
#     ])
# )
# get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
# token_array = jnp.asarray([tokens] * 4)
# token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
# token_array = pz.nx.wrap(token_array, "batch", "seq")
# inputs = llama.inputs.from_basic_segments(token_array)
# get_resids(inputs)[1][0].value.unwrap("batch", "seq", "embedding")[0]

In [10]:
from micrlhf.sampling import sample
sample(llama, tokenizer, "Hi!",
       batch_size=4, do_sample=True)[0]

  0%|          | 0/61 [00:00<?, ?it/s]

(["<bos>Hi! For fun, I changed the foo and also for fun the bar.\nThe FIFO script (not used in this print) is supposed to slowly replace the last 3 items with the first 3 items (OK I know how that isn't FIFO).\nThe code is not tested extensively but it",
  "<bos>Hi!\n\nYour soil sounds dead dry and just because it is green doesn't mean the plants are alive: it is more than half air and water which is way too dry.\n\nBy putting these plants in acidic soil, you are burring them in a deeper chemical death hole. Putting the plants on pine",
  '<bos>Hi!\n\nConsidering the virus spreads faster among vaccinated carriers. (source: The Washington Post, scroll down to "tests don\'t measure "breakthrough" infections")\n\nStarting today, I have noticed that many vaccinated carriers mask everywhere, even with their own child or children who haven\'t received the vaccine yet',
  "<bos>Hi! I am willing to do everything for mama and grandson, but baby Serenity does not get the best. WE want to help reduce the stress and ultimately spoiling that baby Serenity might have. She does NOT get inside outside play time but goes out a few times a month when it's nice out because she is"],
 LlamaKVCachingState(
   cache_len=64,
   batch_axes={'batch': 4},
   kv_caches={'WithSideInputsFromInputTuple.body/WithSideInputsFromInputTuple.body/Sequential.sublayers[3]/Sequential.sublayers[0]/LlamaBlock.sublayers[1]/Residual.delta/Sequential.sublayers[2]/LlamaKVCachingAttention.kv_cache': (<NamedArray bfloat16(| batch:4, projection:256, kv_heads:1, seq:64) ≈0.013 ±1.2 [≥-1.2e+01, ≤7.0] nonzero:65_536 (wrapping jax.Array)>, <NamedArray bfloat16(| batch:4, projection:256, kv_heads:1, seq:64) ≈-0.031 ±1.9 [≥-1.8e+01, ≤1.4e+01] nonzero:65_536 (wrapping jax.Array)>), 'WithSideInputsFromInputTuple.body/WithSideInputsFromInputTuple.body/Sequential.sublayers[3]/Sequential.sublayers[10]/LlamaBlock.sublayers[1]/Residual.delta/Sequential.sublayers[2]/LlamaKVCachingAttention.kv_cache': (<NamedArray bfloat16(| batch:4, projection:256, kv_heads:1, seq:64) ≈0.042 ±1.5 [≥-9.5, ≤1.5e+01] nonzero:65_536 (wrapping jax.Array)>, <NamedArray bfloat16(| batch:4, projection:256, kv_heads:1, seq:64) ≈0.028 ±0.76 [≥-4.7, ≤6.5] nonzero:65_536 (wrapping jax.Array)>), 'WithSideInputsFromInputTuple.body/WithSideInputsFromInputTuple.body/Sequential.sublayers[3]/Sequential.sublayers[11]/LlamaBlock.sublayers[1]/Residual.delta/Sequential.sublayers[2]/LlamaKVCachingAttention.kv_cache': (<NamedArray bfloat16(| batch:4, projection:256, kv_heads:1, seq:64) ≈-0.058 ±1.3 [≥-1.5e+01, ≤7.1] nonzero:65_536 (wrapping jax.Array)>, <NamedArray bfloat16(| batch:4, projection:256, kv_heads:1, seq:64) ≈0.015 ±1.0 [≥-6.8, ≤8.1] nonzero:65_536 (wrapping jax.Array)>), 'WithSideInputsFromInputTuple.body/WithSideInputsFromInputTuple.body/Sequential.sublayers[3]/Sequential.sublayers[12]/LlamaBlock.sublayers[1]/Residual.delta/Sequential.sublayers[2]/LlamaKVCachingAttention.kv_cache': (<NamedArray bfloat16(| batch:4, projection:256, kv_heads:1, seq:64) ≈0.0066 ±1.2 [≥-7.8, ≤1.4e+01] nonzero:65_536 (wrapping jax.Array)>, <NamedArray bfloat16(| batch:4, projection:256, kv_heads:1, seq:64) ≈-0.0016 ±1.2 [≥-7.3, ≤8.2] nonzero:65_536 (wrapping jax.Array)>), 'WithSideInputsFromInputTuple.body/WithSideInputsFromInputTuple.body/Sequential.sublayers[3]/Sequential.sublayers[13]/LlamaBlock.sublayers[1]/Residual.delta/Sequential.sublayers[2]/LlamaKVCachingAttention.kv_cache': (<NamedArray bfloat16(| batch:4, projection:256, kv_heads:1, seq:64) ≈0.028 ±1.4 [≥-1.5e+01, ≤7.2] nonzero:65_536 (wrapping jax.Array)>, <NamedArray bfloat16(| batch:4, projection:256, kv_heads:1, seq:64) ≈0.039 ±1.1 [≥-7.1, ≤6.5] nonzero:65_536 (wrapping jax.Array)>), 'WithSideInputsFromInputTuple.body/WithSideInputsFromInputTuple.body/Sequential.sublayers[3]/Sequential.sublayers[14]/LlamaBlock.sublayers[1]/Residual.delta/Sequential.sublayers[2]/LlamaKVCachingAttention.kv_cache': (<NamedArray bfloat16(| batch:4, projection:256, kv_heads:1, seq:64) ≈0.031 ±

In [7]:
# sample(llama, tokenizer, ["Write me a poem about Machine Learning.", "Hi!"], do_sample=True, return_only_completion=True, seed=3)

In [8]:
# from transformers import AutoModelForCausalLM
# model = AutoModelForCausalLM.from_pretrained("alpindale/gemma-2b")

In [9]:
# import torch
# from collections import OrderedDict
# for m in model.modules():
#     m._forward_hooks = OrderedDict()
# model.model.layers[0].register_forward_hook(lambda self, input, output: print(input[0]))
# tt = torch.LongTensor(tokens).unsqueeze(0)
# model(tt).logits